# Step 6: Data Integration

## Import Libraries

In [1]:
import os
import pandas as pd
import numpy as np
import csv

Here are the columns we need for each unique HDB address. <br>
1. Nearest MRT/LRT station (and its relevant info, eg: MRT service line)
2. Radial distance to nearest MRT station. 
3. Walking distance to nearest MRT station. 
4. Walking time to nearest MRT station. <br><br>
***Within 500m and 1km***
5. Number of MRT/LRT stations nearby.
6. Number of bus stations nearby.
7. Number of schools nearby.
8. Number of food places nearby. 
9. Number of shopping places nearby. 


## User Inputs

In [2]:
outputFile = 'googleOutput_20180319.csv'
nearestMRT = 'nearest_MRT_20180311.csv'
walkingDistance = 'walking_distances_compiled_20180317.csv' # to be replaced. 
MRT = 'MRT.csv'

placesBegin = 'places_2018' 
fileType = 'csv'
placeFilelist = []
for f in os.listdir(): 
    if f[0: len(placesBegin)] == placesBegin and f.split('.')[-1] == fileType:
        placeFilelist.append(f)

# Comment out following line for full run
#placeFilelist = placeFilelist[0:2]     

placeFilelist

['places_20180311.csv',
 'places_20180311_2.csv',
 'places_20180311_3.csv',
 'places_20180311_4.csv',
 'places_20180311_5.csv',
 'places_20180315_1.csv',
 'places_20180315_2.csv',
 'places_20180315_3.csv',
 'places_20180315_4.csv']

## Read Relevant Files

In [3]:
df1 = pd.read_csv(nearestMRT)
df1.set_index('HDB Address', inplace = True)
df2 = pd.read_csv(walkingDistance) 
df2.set_index('HDB Address', inplace = True)
df3 = pd.read_csv(MRT)
df3.set_index('Station Code', inplace = True)

In [4]:
df = df1.loc[:,['Nearest Station Code', 'Vincenty Distance in km']]
df = df.join(df2)

df['Station Match'] = np.where(df['Nearest Station Code'] == df['MRT Station Code'], True, False)
if len(df[df['Station Match'] == False]) != 0:
    print('WARNING: Nearest Station Code from radial distance not matching MRT Station Code from walking distance.')
else:
    df = df.drop('Nearest Station Code', axis = 1)
    
df3 = df3.drop(['Planning Area', 'Region','Connection to other transport means'], axis = 1)
df = df.join(df3, on = 'MRT Station Code')

## Special Treatment for Places

Setup to query the places results file. 

In [5]:
placeTypes = ['bus_station', 'subway_station','cafe','restaurant','school','shopping_mall','supermarket','park']
distance = [1, 0.5]
placeID = {}
placeCount = {}
for d in distance:
    placeID[d] = dict(zip(placeTypes, [{}]*len(placeTypes)))
    placeCount[d] = dict(zip(placeTypes, [{}]*len(placeTypes)))    
placeDirectory = {}

Read places results file into a dictionary

In [6]:
f_t = 0
for f in placeFilelist: # loop for each file
    df4 = pd.read_csv(f) 
    
    for d in distance: # loop for each cut-off distance
        data = df4[df4['Place Distance in km'] <= d]
        
        # create a directory to map place ID to place name as we read the file.
        placeDir = data.loc[:, ['Place ID','PLace Name']].drop_duplicates(inplace = False)
        placeDir = dict(zip(placeDir['Place ID'].tolist(), placeDir['PLace Name'].tolist()))
        placeDirectory.update(placeDir)
        
        t_t = 0
        for t in placeTypes: # loop for each place type
            type_data = data[data['Place Type'] == t]
            
            place = placeID[d][t].copy()
            n = placeCount[d][t].copy()
                        
            index = type_data['HDB Address'].unique().tolist()
            for i in index: # loop for each HDB
                if i in place:
                    place[i] = np.unique(np.append(place[i], type_data.loc[type_data['HDB Address']==i, 'Place ID'].unique()))
                else: 
                    place[i] = type_data.loc[type_data['HDB Address']==i, 'Place ID'].unique()
                n[i] = place[i].shape[0]
                
            placeID[d][t] = place.copy()
            placeCount[d][t] = n.copy()
            t_t +=1
            print('Recorded for %i/%i types of places within %.1f km for File %s. ' %(t_t,len(placeTypes),d,f))
            
        print('Recorded for %.1f km distance for File %s. ' %(d,f))    
        
    f_t +=1
    print('Recorded %i/%i files ' %(f_t, len(placeFilelist)))

Recorded for 1/8 types of places within 1.0 km for File places_20180311.csv. 
Recorded for 2/8 types of places within 1.0 km for File places_20180311.csv. 
Recorded for 3/8 types of places within 1.0 km for File places_20180311.csv. 
Recorded for 4/8 types of places within 1.0 km for File places_20180311.csv. 
Recorded for 5/8 types of places within 1.0 km for File places_20180311.csv. 
Recorded for 6/8 types of places within 1.0 km for File places_20180311.csv. 
Recorded for 7/8 types of places within 1.0 km for File places_20180311.csv. 
Recorded for 8/8 types of places within 1.0 km for File places_20180311.csv. 
Recorded for 1.0 km distance for File places_20180311.csv. 
Recorded for 1/8 types of places within 0.5 km for File places_20180311.csv. 
Recorded for 2/8 types of places within 0.5 km for File places_20180311.csv. 
Recorded for 3/8 types of places within 0.5 km for File places_20180311.csv. 
Recorded for 4/8 types of places within 0.5 km for File places_20180311.csv. 
Reco

Recorded for 8/8 types of places within 0.5 km for File places_20180315_1.csv. 
Recorded for 0.5 km distance for File places_20180315_1.csv. 
Recorded 6/9 files 
Recorded for 1/8 types of places within 1.0 km for File places_20180315_2.csv. 
Recorded for 2/8 types of places within 1.0 km for File places_20180315_2.csv. 
Recorded for 3/8 types of places within 1.0 km for File places_20180315_2.csv. 
Recorded for 4/8 types of places within 1.0 km for File places_20180315_2.csv. 
Recorded for 5/8 types of places within 1.0 km for File places_20180315_2.csv. 
Recorded for 6/8 types of places within 1.0 km for File places_20180315_2.csv. 
Recorded for 7/8 types of places within 1.0 km for File places_20180315_2.csv. 
Recorded for 8/8 types of places within 1.0 km for File places_20180315_2.csv. 
Recorded for 1.0 km distance for File places_20180315_2.csv. 
Recorded for 1/8 types of places within 0.5 km for File places_20180315_2.csv. 
Recorded for 2/8 types of places within 0.5 km for File 

Convert dictionaries into columns.

In [7]:
columns={}
index = list(df.index)

for d in distance:
    for t in placeTypes:
        column = {}
        for i in df.index:
            if i not in placeCount[d][t]:
                column[i] = 0
            else:
                column[i] = placeCount[d][t][i]
        columns['n_' + t + '_' + str(d) + '_km']=column

In [8]:
for c in columns:
    df[c] = pd.Series(columns[c])

In [9]:
df.to_csv(outputFile)

In [10]:
df.head()

,Nearest Station Code,Vincenty Distance in km,MRT Station Code,Walking Distance in Meters,Walking Duration in Minutes,Station Match,Station Name,NS,EW,CG,...,n_supermarket_1_km,n_park_1_km,n_bus_station_0.5_km,n_subway_station_0.5_km,n_cafe_0.5_km,n_restaurant_0.5_km,n_school_0.5_km,n_shopping_mall_0.5_km,n_supermarket_0.5_km,n_park_0.5_km
HDB Address,,,,,,,,,,,,,,,,,,,,,
Blk+83+COMMONWEALTH+CL+Singapore,EW20,0.356413,EW20,792,9.883333,True,Commonwealth,False,True,False,...,7,3,10,1,25,39,9,0,2,1
Blk+97+COMMONWEALTH+CRES+Singapore,EW20,0.544454,EW20,828,10.516667,True,Commonwealth,False,True,False,...,6,2,8,0,21,33,5,0,1,0
Blk+95+COMMONWEALTH+DR+Singapore,EW20,0.463413,EW20,725,9.283333,True,Commonwealth,False,True,False,...,6,2,10,1,23,35,6,0,1,0
Blk+98+COMMONWEALTH+CRES+Singapore,EW20,0.608135,EW20,812,10.350000,True,Commonwealth,False,True,False,...,6,2,8,0,21,33,5,0,1,0
Blk+93+COMMONWEALTH+DR+Singapore,EW20,0.383429,EW20,610,7.950000,True,Commonwealth,False,True,False,...,7,2,9,1,24,37,11,0,2,1


## Test Codes

In [11]:
i = 'Blk+95+COMMONWEALTH+DR+Singapore'
d  = 1
t = 'bus_station'
name = 'n_' + t + '_' + str(d) + '_km'

print(name)
print(placeCount[d][t][i])
print(columns[name][i])

n_bus_station_1_km
39
39


In [12]:
# to see the actual names of places nearby a HDB on columns. 
i = 'Blk+93+COMMONWEALTH+DR+Singapore'
d  = 0.5
print('Places within %.1f km from %s' %(d,i))
for t in placeID[d]:
    if i in placeID[d][t]:
        print('==========================%i %s==========================' %(placeCount[d][t][i],t))
        for p in placeID[d][t][i]:
            print(p, placeDirectory[p])
    else:
        print('==========================%i %s==========================' %(0,t))
    print('==============================================================')

Places within 0.5 km from Blk+93+COMMONWEALTH+DR+Singapore
==========================9 bus_station==========================
ChIJefkNpD8a2jERvwCryGTgxno Blk 92
ChIJW617-T4a2jERse0GM8yi2GQ Opp Blk 95
ChIJ7e2KUT4a2jER2OpCTN9anSk Blk 97
ChIJkycfoEAa2jERHcZ1WoizU0M Opp Blk 115B
ChIJAUlR2Dga2jERUsfPSEMfT-4 Opp Queensway Sec Sch
ChIJpZZuWEca2jERnuoAA0IWrjM C'wealth Stn
ChIJYyJzITka2jERXDjdqjOsbkU Queensway Sec Sch
ChIJwR49YEca2jER8T1MMddBQGU C'wealth Stn
ChIJafWljRUa2jERryMl0vEMZzk Viz Holland
==========================1 subway_station==========================
ChIJhY0kWUca2jERcQr3RFss0Fg Commonwealth MRT Station
==========================24 cafe==========================
ChIJPXz6sD8a2jER5AjLxp8r_Qg Noodles Stove
ChIJg7jivT8a2jERMiC0yS_yAd0 Eng Kee: Famous Chicken Wing & Fried Bihun
ChIJO3wUuD8a2jER2zMYHJRHrZU Big Daddy Chicken Rice
ChIJAaaCuD8a2jERkAoBlfPUteY Cahaya Malay Food Stall
ChIJl1w9uj8a2jERPJDhVM5bzF0 Hotspot western
ChIJf866wD8a2jERCH9N47_Wttg Selera Tiga Puteri Malay Food Stall
C